In [2]:
from muticore_search import *

In [41]:
def muticore_findDoc(query,processorNum=6):
    start = time()
    totalParts = 47
    ranges = divide(totalParts, processorNum)

    p = Pool(processorNum)
    for id,partsRange in enumerate(ranges):
        p.apply_async(sub_process, args=(id,partsRange,query))
    #print('Waiting for all subprocesses done...')
    p.close()
    p.join()
    #print('All subprocesses done.')

    result = []
    for id in range(processorNum):
        with open(result_path(id),encoding='utf8') as f:
            for line in f.readlines():
                result.append(line.strip('\n'))
    #print('找到{}个相关页面，用时 {} s'.format(len(result),time()-start))
    return(result)

In [42]:
query = '中华人民共和国共产党的领导人是谁'
result = muticore_findDoc(query,5)
result[:10]

['pages/0/0/2',
 'pages/0/0/46',
 'pages/0/0/80',
 'pages/0/0/81',
 'pages/0/0/98',
 'pages/0/0/104',
 'pages/0/0/149',
 'pages/0/0/151',
 'pages/0/0/152',
 'pages/0/0/156']

In [43]:
from langconv import *

line = '為当今世界著名的[[美國]][[網球]]女子巨星，網球史上最伟大的女子选手之一，\
    美國體育史上最佳女子網球选手（與[[克里斯·埃弗特|Chris Evert]]並列），\
    全世界第一位網球大滿貫黑人女單冠軍，單打最高世界排名第一，\
    23座大滿貫女單冠軍得主，網球史上第二位女子單打生涯全滿貫得主。'
# 转换繁体到简体
line = Converter('zh-hans').convert(line)
print(line)
# 转换简体到繁体
line = Converter('zh-hant').convert(line)
print(line)

为当今世界著名的[[美国]][[网球]]女子巨星，网球史上最伟大的女子选手之一，    美国体育史上最佳女子网球选手（与[[克里斯·埃弗特|Chris Evert]]并列），    全世界第一位网球大满贯黑人女单冠军，单打最高世界排名第一，    23座大满贯女单冠军得主，网球史上第二位女子单打生涯全满贯得主。
為當今世界著名的[[美國]][[網球]]女子巨星，網球史上最偉大的女子選手之一，    美國體育史上最佳女子網球選手（與[[克裡斯·埃弗特|Chris Evert]]併列），    全世界第一位網球大滿貫黑人女單冠軍，單打最高世界排名第一，    23座大滿貫女單冠軍得主，網球史上第二位女子單打生涯全滿貫得主。


In [50]:
import jieba
stopWords = ['上','的','什么','什麼','是','大','第几',\
             '过','多少','多少次','？','哪','谁','哪里',\
            '哪个','哪种','哪位','哪座','指','和']

def findLine(query, minLen=10):
    s = []
    docList = muticore_findDoc(query)
    if(len(docList) > 100): #如果搜索到的文档太多，则随机选出100篇
        docList = random.sample(docList,100)
    keywords = [word for word in jieba.cut_for_search(query)]

    for doc in docList:
        with open(doc,encoding='utf8') as f:
            for line in f.readlines():
                if(len(line) > minLen):
                    tf = {}
                    for word in jieba.cut_for_search(line):
                        if(word in keywords and word not in stopWords):
                            tf[word] = 1 
                    bingo = len(tf)
                    s.append(('位于 '+doc+'   '+line,bingo))
    return s

def pickAns(query, minLen=10, maxReturn=1):
    s = []
    s.extend(findLine(query, minLen))
    query = Converter('zh-hant').convert(query)
    s.extend(findLine(query,minLen))                
    s = sorted(s,key=lambda x:x[1], reverse=1)
    return [item[0] for item in s[:maxReturn]]

In [51]:
questions = []
with open('wdm_assignment_3_samples.txt',encoding='utf8') as fin:
    for line in fin.readlines():
        questions.append(line.split('\t')[0])

questions[:10]

['马尔代夫的第一大支柱产业是什么？',
 '澳大利亚是南半球面积第几大的国家？',
 '阿根廷国家足球队赢得过多少次美洲杯冠军？',
 '平均人口密度最高的大洲是？',
 '中国历史上最杰出的浪漫主义诗人是？',
 '甲午战争爆发的标志是？',
 '《华英字典》的作者是？',
 '《资治通鉴》的撰写一共耗时多少年？',
 '世界上首位从南坡登上珠穆朗玛峰的女性的国籍是？',
 '最后宣布废除奴隶制度的行政地区是？']

In [ ]:
for q in questions:
    print('问:',q)
    print('答:',pickAns(q,maxReturn=1))
    print()

问: 马尔代夫的第一大支柱产业是什么？
答: ['位于 pages/0/33/56   马尔代夫是个岛国，陆地面积相当狭窄。[[工业]]、[[农业]]水平低下。[[渔业]]、[[航运]]和[[旅游]]是三大经济支柱。近年来，随着旅游业的发展，旅游收入已达到了国民总产值的70%以上。成为马尔代夫的第一支柱产业。\n']

问: 澳大利亚是南半球面积第几大的国家？
答: ["位于 pages/0/2/185   = 1-876429-37-2|date=2003}}</ref><ref>{{cite web |title = Australia |work=Oxford Dictionaries |date=April 2010 |publisher=Oxford University Press |url = http://oxforddictionaries.com/definition/english/Australia |accessdate=2012-07-26}}</ref>，[[缩写]]为'''AU'''、'''AUS'''），中文环境下（尤其是臺灣、香港等地）常使用「'''澳-{}-洲'''」代替“-{zh-cn:澳大利亚;zh-tw:澳大利亞}-”，是全球[[面积]]第六大的[[国家]]，[[大洋洲]]最大的国家和[[南半球]]第二大的国家。澳大利亚国土包括澳大利亚大陆，[[塔斯曼尼亚|塔斯曼尼亚岛]]及数个海外的岛屿，面积和[[美國本土]]相似，是[[世界]]上唯一一个国土覆盖整个[[大陆]]的国家。與隔海相望的东南近邻是[[紐西蘭]]，西北是[[印度尼西亚]]，北边是[[巴布亚新几内亚]]、[[西巴布亚]]及[[东帝汶]]。\n"]

问: 阿根廷国家足球队赢得过多少次美洲杯冠军？
答: ['位于 pages/1/126/255   到了1980年代，“王子”[[恩佐·弗朗西斯科利|弗朗西斯科利]]、“灵猫”[[鲁本·索萨|索萨]]、本戈切亚、丰塞卡、波耶特等一批球星的出现使乌拉圭足球有了转机。1980年底到1981年初，为纪念世界杯创办50周年，乌拉圭举办了[[1980-81年世界冠軍金盃足球賽|世界金杯足球赛]]，邀请历届世界杯冠军得主齐聚一堂，这被认为是后来[[联合会杯]]的雏形。意大利、西德、巴西、阿根廷应邀参赛，惟英格兰拒绝参加，改由兩届

答: ["位于 pages/7/184/45   *[[1965年]]在[[云南]]发现了'''[[元谋人|什么古人类]]'''化石？\n"]

问: 美国的当代人类学通常划分为几大分支？
答: []

问: “中华民族”一词最早由谁提出？
答: ['位于 pages/0/1/167   \'\'\'中國人\'\'\'（{{lang-en|Chinese people}}{{notetag|英文“Chinese”具有多個涵義，<!--其中{{fact|“Chinese People”一般指出生在中国的[[中国公民]]，包括汉族及55个少数民族，但不包括外籍华人。}}要{{fact|表达“Ethnic Chinese”时，可使用[[華人]]一词，}}{{fact|“Ethnic Chinese”主要指具有[[汉族]]血统的人，}}而其他-->[[中国少数民族]]如：[[維吾爾族]]（[[维吾尔族]]）、[[朝鮮族]]（[[朝鲜族]]）、[[蒙古族]]（[[蒙古族]]）、[[中国俄罗斯族]]（[[中国俄罗斯族]]）等在英文中一般不称“Chinese”，而用其专有的民族名称。}}），可以有多重含义：以「[[中國地理|地理]]」概念界定是指出生或居住在[[中國]]的人；以「[[國籍]]」概念界定是指具有[[中國國籍]]的人<ref>[[s:中華人民共和國憲法]]</ref>；以「[[国族]]」概念来界定是指属于[[中华民族]]的人，以[[族群]]的概念来界定一般指汉族人或者是说汉语的少数民族。<ref>孙中山：[[三民主義]]</ref>。“中國人”一詞开始普遍使用，主要是在1912年[[中華民國]]建國之後<ref name="王賡武黃堅立2002">{{cite book|author1=王賡武|author2=黃堅立|title=海外華人硏究的大視野與新方向: 王賡武敎授論文選|url=http://books.google.com/books?id=KxcafX3JggYC&pg=PA100|accessdate=19 January 2013|date=June 2002|publisher=八方文化创作室|isbn=978-1-879771-61-1|pages=100–|quote=一般來講，我看中國人這個詞彙普偏地使用，還是二十世紀之後用的比較多一些}}</

答: ['位于 pages/0/72/130   卢旺达议会实行[[两院制]]，宪法赋予议会立法权，并且可以监督总统和内阁的运作<ref name="CJCR-2003" />{{rp|62}}。其中的[[下议院]]是卢旺达众议院，拥有80位议员，任期5年。所有议席中有24个保留给女性，这些女性议员都是通过地方政府官员组成的联合大会选举产生；另外还有三个议席为青少年和残疾人保留，剩下的53个席位则按[[比例代表制]]经[[普通选举]]产生<ref name="CJCR-2003" />{{rp|76}}。2008年的议会选举共计产生了45位女性众议员，卢旺达也因此成为全世界唯一一个女性在全国性议会中占据多数的国家<ref name="UNIFEM-2008">{{cite news | author = United Nations Development Fund for Women (UNIFEM) | date = 2008-09-22 | title = Rwandan Women Secure 56% of Parliamentary Seats in Historic Election Result | url = http://www.unifem.org/news_events/story_detail882c.html | accessdate = 2014-04-16 | deadurl = no | archivedate = 2013-12-13 | archiveurl = https://web.archive.org/web/20131213053702/http://www.unifem.org/news_events/story_detail882c.html}}</ref>。卢旺达参议院是两院中的[[上议院]]，通过各种团体选出，至少30%的参议员必须是女性，参议员的任期为八年<ref name="CJCR-2003" />{{rp|82}}。\n']

问: 中国近代第一所国立综合性大学是？
答: ['位于 pages/0/0/175   至近代，[[中國教育史|新式教育]]興起，[[國子監]]和[[科舉制度]]被取消，而1898年中国近代第一所国立综合性大学[[京師大學堂]]（今[[北京大學]]）在北京成立<ref>趙爾巽等：

答: []

问: 人类最早发现的抗生素的发现者在哪一年获得了诺贝尔生理医学奖？
答: []

问: 牛津大学培养了几位获得过菲尔兹奖的数学家？
答: []

问: 世界四大长河是？
答: ['位于 pages/0/6/251   \'\'\'洛阳市\'\'\'（{{Lang-pinyin|\'\'\'Luòyáng\'\'\'}}，[[邮政式拼音]]：\'\'\'Loyang\'\'\'）是[[中华人民共和国]][[河南省]]下辖的[[地级市]]，位于[[中原]]地区，中国[[四大古都]]之一，现为河南省及[[中原城市群]]的副中心城市。是[[丝绸之路]]起点，[[隋唐大运河]]的中心城市，也是[[中华人民共和国国务院|国务院]]认定的首批24个[[国家历史文化名城]]之一，2011年被[[联合国]]授予“世界文化名城”称号<ref name="sjmc"/>。洛阳自夏朝开始，前后有十三个正统朝代以其为都，是中国历史上建都最早、建都政权数量最多、持续建都时间最久的都城，拥有5000年文明史、3000年建城史和1500年建都史。仰韶曙光，夏商肇基，周制礼乐，两汉文章，魏晋风流，南北融合，隋唐盛世，宋金兵戈，洛阳城在历史的长河中数度兴废，活跃在文明中心长达数千年，与国家与民族的命运始终相依。<ref>{{cite web|url=http://www.hnjjbs.com/Recommend/2010-12-10/3134.html|title=洛阳：打造河南省副中心城市|publisher=河南经济报|date=2010-12-10|language =zh-hans }}{{dead link|date=January 2014}}</ref>\n']

问: 金庸的封笔之作是？
答: ['位于 pages/8/236/134   * 明清之交的名人[[查继佐]]是金庸先生的祖先之一，此人在《聊斋志异》与《觚胜雪莲》中有记载，蒲松龄在其作品中赞扬此人“厚施而不问其名，真侠烈古丈夫哉！”。《觚胜雪莲》中记述查继佐“才华丰艳，而风情潇洒。常谓满眼悠悠，不堪酬对；海内奇杰，非从尘埃中物色，未可得也”。金庸先生在其封笔之作《鹿鼎记》当中也曾提及此人生平。然真實情況卻並非如此。《查繼佐年譜》裡，查繼佐本人亦否認有此事，說：「葛如，方布衣野走，世傳余有一飯之恩，懷之而思報。其

答: []

问: 文艺复兴的发源于意大利的哪个城市？
答: ["位于 pages/10/249/97   '''文艺复兴美术'''，是文艺复兴时代欧洲美术风格的统称。是整个[[文艺复兴运动]]的一部分，大致发展阶段是在14世纪兴起，15世纪全面展开，16世纪达到了鼎盛期<ref>{{cite book zh|author=徐庆平 著|title=《意大利文艺复兴美术》|pages=第7页|ISBN=7-300-05318-1}}</ref>。17世纪后，欧洲美术风格渐渐转向[[矫饰主义]]。文艺复兴美术是西欧[[城市文化]]繁荣的产物，同时它与[[教会]]关系密切。在绘画上的成就最大，建筑和雕刻则稍逊之。文艺复兴美术最初发源于意大利的[[佛罗伦萨]]，随后逐渐向欧洲各地传播，在[[意大利]]和[[德国]]、[[尼德兰]]尤甚，在这些地区都产生自己杰出的艺术家。文艺复兴美术在南欧和北欧都有自己的特点，作为文艺复兴美术中心的意大利，成为后来几个世纪里，欧洲艺术家们的朝圣地。这个时期，艺术人才辈出，特别是[[列奥纳多·达·芬奇|达芬奇]]、[[米开朗基罗]]、[[拉斐尔·圣齐奥|拉斐尔]]被誉为[[文艺复兴三杰]]。\n"]

问: 西班牙国家德比的两支球队一共夺得过多少次西甲冠军？52

答: []

问: 莱昂纳多第一次获得奥斯卡提名是多少岁？
答: []

问: 唯一获得香港电影金像奖最佳女配角的中国大陆演员是？
答: ['位于 pages/0/251/64   2008年4月，李宇春发行独立企划迷你专辑《少年中国》。歌曲“少年中国”荣获第6届东南劲爆音乐榜\xa0<ref name=":1">{{Cite web|url=http://ent.sina.com.cn/y/2008-11-15/16522251566.shtml|title=第六届东南劲爆音乐榜颁奖礼获奖名单|accessdate=2016年10月28日|author=|date=2008年11月15日|publisher=新浪网}}</ref>、第7届MusicRadio中国TOP排行榜、中国原创音乐流行榜总选“年度金曲”。李宇春荣获MusicRadio中国TOP排行榜<ref>{{Cite web|url=http://ent.people.com.cn/GB/9197136.html|tit